## **Installing libraries**

In [1]:
!pip install --no-cache-dir "numpy==1.24.4" "qiskit==0.45.1" "qiskit-algorithms==0.2.1" "qiskit-optimization==0.6.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 27.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 204.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 255.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 290.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.9/306.9 kB 328.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 284.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 241.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 259.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### **Using Quantum Optimization**

In [1]:
from qiskit import QuantumCircuit
from qiskit_algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_optimization.applications import GraphOptimizationApplication
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.circuit.library import TwoLocal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import time

In [3]:
data = pd.read_csv('/content/student_timetable.csv', encoding = "latin1")
data.head()

,student_id,year_semester,course_id,course,weekday,start_time,end_time,room_address
0,100115,20191,192015,Primary Teaching,Monday,08:00,11:40,FE-5 - SALA DE AULA 04
1,100115,20192,142166,Supervised Internship in French 1,Tuesday,16:00,17:50,Missing info
2,100115,20192,142166,Supervised Internship in French 1,Thursday,16:00,17:50,Missing info
3,100140,20191,139190,Social History And General Policy,Tuesday,16:00,17:50,SALA PAT AT 045 ...
4,100140,20191,134911,Political Sociology,Wednesday,14:00,15:50,SALA PAT AT 061 ...


In [4]:
print("Original dataset shape:", data.shape)
print(data.head())

Original dataset shape: (100473, 8)
   student_id  year_semester  course_id                             course  \
0      100115          20191     192015                   Primary Teaching   
1      100115          20192     142166  Supervised Internship in French 1   
2      100115          20192     142166  Supervised Internship in French 1   
3      100140          20191     139190  Social History And General Policy   
4      100140          20191     134911                Political Sociology   

     weekday start_time end_time  \
0     Monday      08:00    11:40   
1    Tuesday      16:00    17:50   
2   Thursday      16:00    17:50   
3    Tuesday      16:00    17:50   
4  Wednesday      14:00    15:50   

                                        room_address  
0                            FE-5  - SALA DE AULA 04  
1                                       Missing info  
2                                       Missing info  
3  SALA PAT AT 045                               ...  
4  

In [5]:
reduced_data = data.head(10)

In [7]:
students = reduced_data['student_id'].unique()
courses = reduced_data['course'].unique()
print("Students:", students)
print("Courses:", courses)

Students: [100115 100140]
Courses: ['Primary Teaching' 'Supervised Internship in French 1'
 'Social History And General Policy' 'Political Sociology'
 'Class Structure And Social Stratification'
 'Contemporary Sociological Theories 1']


### **Conflict Matrix**

In [9]:
# build Conflict Matrix
conflict_matrix = pd.DataFrame(0, index=courses, columns=courses)

# Mark conflicts: if 2 courses share a student
for student in students:
    enrolled_courses = reduced_data[reduced_data['student_id'] == student]['course'].tolist()
    for i in range(len(enrolled_courses)):
        for j in range(i+1, len(enrolled_courses)):
            c1, c2 = enrolled_courses[i], enrolled_courses[j]
            conflict_matrix.loc[c1, c2] += 1
            conflict_matrix.loc[c2, c1] += 1

print("\nConflict Matrix:\n", conflict_matrix)


Conflict Matrix:
                                            Primary Teaching  \
Primary Teaching                                          0   
Supervised Internship in French 1                         2   
Social History And General Policy                         0   
Political Sociology                                       0   
Class Structure And Social Stratification                 0   
Contemporary Sociological Theories 1                      0   

                                           Supervised Internship in French 1  \
Primary Teaching                                                           2   
Supervised Internship in French 1                                          2   
Social History And General Policy                                          0   
Political Sociology                                                        0   
Class Structure And Social Stratification                                  0   
Contemporary Sociological Theories 1                       

In [11]:
qp = QuadraticProgram()

# Each course is assigned to one of 3 time slots
timeslots = range(3)
course_vars = {}

for course in courses:
    for t in timeslots:
        var_name = f"{course}_{t}"
        qp.binary_var(name=var_name)
        course_vars[(course, t)] = var_name

# Constraint: one time slot per course
for course in courses:
    qp.linear_constraint(
        linear={f"{course}_{t}": 1 for t in timeslots},
        sense='==',
        rhs=1,
        name=f"one_slot_{course}"
    )

# Objective: minimize conflicts
penalty = 1.0
objective = {}

for c1 in courses:
    for c2 in courses:
        if conflict_matrix.loc[c1, c2] > 0 and c1 != c2:
            for t in timeslots:
                var1 = course_vars[(c1, t)]
                var2 = course_vars[(c2, t)]
                key = (var1, var2)
                if key in objective:
                    objective[key] += penalty
                else:
                    objective[key] = penalty

qp.minimize(quadratic=objective)

In [15]:
from qiskit_algorithms.optimizers import COBYLA

In [20]:
import time

start_time = time.time()

eigensolver = NumPyMinimumEigensolver()
optimizer = MinimumEigenOptimizer(eigensolver)
result = optimizer.solve(qp)

end_time = time.time()
print("\n--- Classical QUBO Solution (NumPyMinimumEigensolver) ---")
print(result)


--- Classical QUBO Solution (NumPyMinimumEigensolver) ---
fval=2.0, Primary Teaching_0=0.0, Primary Teaching_1=0.0, Primary Teaching_2=1.0, Supervised Internship in French 1_0=0.0, Supervised Internship in French 1_1=1.0, Supervised Internship in French 1_2=0.0, Social History And General Policy_0=0.0, Social History And General Policy_1=0.0, Social History And General Policy_2=1.0, Political Sociology_0=1.0, Political Sociology_1=0.0, Political Sociology_2=0.0, Class Structure And Social Stratification_0=1.0, Class Structure And Social Stratification_1=0.0, Class Structure And Social Stratification_2=0.0, Contemporary Sociological Theories 1_0=0.0, Contemporary Sociological Theories 1_1=1.0, Contemporary Sociological Theories 1_2=0.0, status=SUCCESS


In [21]:
print("\nTime taken: %.2f seconds" % (end_time - start_time))


Time taken: 1.23 seconds


In [39]:
# Extract the assignment from the result
assignments = result.x
variable_names = [var.name for var in qp.variables]

# Create a list to store the course assignments for plotting
plot_data = []
for course in courses:
    for t in timeslots:
        var_name = course_vars[(course, t)]
        try:
            var_index = variable_names.index(var_name)
            if assignments[var_index] > 0.5:
                plot_data.append({'Course': course, 'Time Slot': f'Slot {t+1}'})
        except ValueError:
            print(f"Warning: Variable name {var_name} not found in qp.variables")


# Create a DataFrame for plotting
plot_df = pd.DataFrame(plot_data)
fig = px.bar(plot_df, x='Time Slot', y='Course', title='Course Assignments per Time Slot')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [40]:
# Display the optimized timetable
optimized_timetable = pd.DataFrame(plot_data)
print("\nOptimized Timetable:")
display(optimized_timetable)


Optimized Timetable:


,Course,Time Slot
0,Primary Teaching,Slot 3
1,Supervised Internship in French 1,Slot 2
2,Social History And General Policy,Slot 3
3,Political Sociology,Slot 1
4,Class Structure And Social Stratification,Slot 1
5,Contemporary Sociological Theories 1,Slot 2


## **Solving the Problem using simulated annealing**


In [42]:
import random

# Define the number of time slots
num_timeslots = 3

# Initialize a random timetable state
# The state is a dictionary where keys are courses and values are assigned time slots
initial_state = {course: random.randint(0, num_timeslots - 1) for course in courses}

print("Initial Timetable State:")
print(initial_state)

Initial Timetable State:
{'Primary Teaching': 2, 'Supervised Internship in French 1': 0, 'Social History And General Policy': 2, 'Political Sociology': 0, 'Class Structure And Social Stratification': 0, 'Contemporary Sociological Theories 1': 0}


## **Implement the cost function**



In [49]:
def calculate_cost(timetable):
    cost = 0
    course_list = list(timetable.keys())
    for i in range(len(course_list)):
        for j in range(i + 1, len(course_list)):
            c1 = course_list[i]
            c2 = course_list[j]
            if conflict_matrix.loc[c1, c2] > 0 and timetable[c1] == timetable[c2]:
                cost += 1
    return cost

# usage with the initial random state
cost = calculate_cost(initial_state)
print(f"Initial cost: {cost}")

Initial cost: 3


## **Implement the neighbor generation function**


In [44]:
def get_neighbor(timetable):
    """Generates a neighboring timetable state by randomly reassigning one course's time slot."""
    neighbor_timetable = timetable.copy()
    course_to_change = random.choice(list(neighbor_timetable.keys()))
    current_slot = neighbor_timetable[course_to_change]
    new_slot = random.randint(0, num_timeslots - 1)
    while new_slot == current_slot:
        new_slot = random.randint(0, num_timeslots - 1)
    neighbor_timetable[course_to_change] = new_slot
    return neighbor_timetable

# Example usage:
neighbor_state = get_neighbor(initial_state)
print("\nNeighbor Timetable State:")
print(neighbor_state)


Neighbor Timetable State:
{'Primary Teaching': 1, 'Supervised Internship in French 1': 0, 'Social History And General Policy': 2, 'Political Sociology': 0, 'Class Structure And Social Stratification': 0, 'Contemporary Sociological Theories 1': 0}


## **Implement the simulated annealing algorithm**


In [48]:
def simulated_annealing(initial_state, cost_function, neighbor_function, initial_temperature, cooling_rate, num_iterations):

    current_state = initial_state
    best_state = initial_state.copy()

    current_cost = cost_function(current_state)
    best_cost = current_cost

    for i in range(num_iterations):
        temperature = initial_temperature * (cooling_rate ** i)
        neighbor_state = neighbor_function(current_state)
        neighbor_cost = cost_function(neighbor_state)

        delta_cost = neighbor_cost - current_cost

        if delta_cost < 0:
            current_state = neighbor_state
            current_cost = neighbor_cost
        else:
            acceptance_probability = np.exp(-delta_cost / temperature)
            if random.random() < acceptance_probability:
                current_state = neighbor_state
                current_cost = neighbor_cost

        if current_cost < best_cost:
            best_state = current_state.copy()
            best_cost = current_cost


    return best_state

# Define parameters for simulated annealing
initial_temperature = 100.0
cooling_rate = 0.99
num_iterations = 10000

# Run simulated annealing
start_time_sa = time.time()
best_timetable_sa = simulated_annealing(initial_state, calculate_cost, get_neighbor, initial_temperature, cooling_rate, num_iterations)
end_time_sa = time.time()

best_cost_sa = calculate_cost(best_timetable_sa)

print("\n--- Simulated Annealing Solution ---")
print("Best Timetable State:", best_timetable_sa)
print("Best Cost:", best_cost_sa)
print("Time taken: %.2f seconds" % (end_time_sa - start_time_sa))


--- Simulated Annealing Solution ---
Best Timetable State: {'Primary Teaching': 2, 'Supervised Internship in French 1': 0, 'Social History And General Policy': 2, 'Political Sociology': 0, 'Class Structure And Social Stratification': 1, 'Contemporary Sociological Theories 1': 0}
Best Cost: 1
Time taken: 1.55 seconds


In [46]:
# Create a list to store the course assignments for plotting
plot_data_sa = []
for course, slot in best_timetable_sa.items():
    plot_data_sa.append({'Course': course, 'Time Slot': f'Slot {slot+1}'})

# Create a DataFrame for displaying the timetable
optimized_timetable_sa = pd.DataFrame(plot_data_sa)

# Print a header and display the DataFrame
print("\nOptimized Timetable (Simulated Annealing):")
display(optimized_timetable_sa)


Optimized Timetable (Simulated Annealing):


,Course,Time Slot
0,Primary Teaching,Slot 1
1,Supervised Internship in French 1,Slot 3
2,Social History And General Policy,Slot 3
3,Political Sociology,Slot 2
4,Class Structure And Social Stratification,Slot 1
5,Contemporary Sociological Theories 1,Slot 3


In [52]:
# Create a DataFrame for time comparison
time_comparison_data = {
    'Method': ['Classical QUBO', 'Simulated Annealing'],
    'Time (seconds)': [end_time - start_time, end_time_sa - start_time_sa]
}
time_comparison_df = pd.DataFrame(time_comparison_data)

# Create a bar chart for time comparison
fig_time = px.bar(time_comparison_df, x='Method', y='Time (seconds)', title='Time Comparison: Classical QUBO vs. Simulated Annealing')
fig_time.update_layout(yaxis_title='Time (seconds)')
fig_time.show()